<a href="https://colab.research.google.com/github/secutron/Practice_Ignite/blob/main/A_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HandlersTimeProfiler

##A.1. 속성 예제 

 때로는 역순으로, 완결 코드로부터 시작하여 궁금한 부분만 선택적으로 확인하는 것이 더 효율적일 때가 있다. 이를 위해 이그나이트([ignite](https://pytorch.org/ignite/))의 [auto_dataloader](https://pytorch.org/ignite/v0.4.6/generated/ignite.distributed.auto.auto_dataloader.html#auto-dataloader)와 [auto_model](https://pytorch.org/ignite/v0.4.6/generated/ignite.distributed.auto.auto_model.html#ignite.distributed.auto.auto_model), [auto_optim](https://pytorch.org/ignite/v0.4.6/generated/ignite.distributed.auto.auto_optim.html#ignite.distributed.auto.auto_optim), 그리고 트레이너(trainer)와 이벤트 핸들러(event handler)등의 중요한 부분을  포함하는, 구글 colab 기반의 속성 예제 코드를 아래와 같이 준비하였다.

 그리고 이그나이트([ignite](https://pytorch.org/ignite/))는 작성 시점을 기준으로 다음의 커뮤니케이션 백엔드들을 지원한다.

- backends from native torch distributed configuration: “nccl”, “gloo”, “mpi”
- XLA on TPUs via pytorch/xla
- using Horovod framework as a backend

*커뮤니케이션 백엔드: 상기 ‘’’1.4 분산 딥러닝 기본 지식’’’ 항목에서 언급된 바와 같이 분산처리를 위해서는 컴퓨팅 코어 간 통신이 필요하며, 만일 이 컴퓨팅 코어가 CPU인 경우 gloo나 mpi 백엔드, GPU 경우에는 nccl 백엔드, TPU 경우에는 xla 백엔드 이용이 가능하다. 이와 관련한 보다 상세한 내용은 DISTRIBUTED COMMUNICATION PACKAGE - TORCH.DISTRIBUTED 페이지의 [rule of thumb](https://pytorch.org/docs/stable/distributed.html) 항목을 참조한다. ---~~분산처리의 역사만큼이나 다양한 분산처리 방식이 존재한다. TCP 등을 이용해 직접 프로세스간 통신을 처리하는 방법도 있지만, 하이레벨 관점에서 CPU는 gloo, GPU는 nccl, TPU는 xla 백엔드를 사용해야 한다고 생각하는 것이 정신건강에 좋다.~~---

###A.1.1. Colab runtime 설정

 분산처리에 익숙하지 않은 개발자의 경우, 구글 colab을 이용하여 DDL(Distributed Deep Learning)의 기초를 연습하는 것이 여러 면에서 장점이 있다. 우선 가장 중요한 것은 CPU와 GPU, 그리고 TPU 환경을 쉽게 번갈아가며 분산처리 코드의 동작 상태를 확인할 수 있다는 점이다. ~~그리고 아직 colab은 무료이다. 럭셔리한 환경이라면 GCP를 사용해도 좋겠지.~~ 

 다만 아쉽게도 현 시점에서 colab 무료 버전을 사용하는 경우, 멀티 GPU(Multi-GPU)나 멀티 노드(Multi-Node) 환경은 테스트할 수 없다. 하지만 이그나이트([ignite](https://pytorch.org/ignite/))에서는 작성한 단일 코드를 다양한 환경에서 동일하게 사용할 수 있으므로, 가능한 편안한 환경에서 코드 개발을 선행하고 이후 멀티노드와 같은 목표 환경으로 옮기는 작업 방식을 추천한다. 

  Colab에서는 아래 그림에서와 같이 런타임 환경 설정을 통해 CPU와 GPU, TPU 사용을 선택할 수 있다. ~~하드웨어 가속기가 None이면 CPU만 있다는 뜻~~

 <div align="center">
<img width=512 src="https://i.imgur.com/gcm9MlH.png"/>
</div>

 런타임 변경이 발생할 때마다 새로운 VM(Virtual Machine)이 할당되며, 기존 노트 셀(cell)을  실행시킨 내용이 모두 사라지게 되므로 이에 주의한다. ---~~코드 수정이 사라지는  것이 아니라 셀 실행한 내용이 사라지게 되는 것이다. 예를 들어 셀 실행을 통해 학습 데이터를 VM에 내려받은 경우, 런타임 변경을 통해 새롭게 VM이 할당될 경우 기존 내려받은 데이터는 당연히 새로운 VM에서 찾아볼 수 없다.~~---

  그리고 TPU를 통해 분산처리를 진행하는 경우, 특별히 colab runtime을 TPU 구동이 가능한 VM으로 환경을 설정해주는 작업이  필요하다. 이를 위해 아래와 같은 코드로 colab의 runtime type이 TPU인지 먼저 확인한다. 그리고 검출된 환경이 TPU인 경우, 분산처리가 가능한 런타임으로 변경하는 작업을 수행한다. 참고로 GPU 사용 시에는 런타임 유형 변경에서 GPU를 선택하는 것만으로 GPU를 사용할 수 있다. 

 다행히 파이토치 커뮤니티에서는 아래와 같이 사용이 편리한 xla 설정 관련 스크립트를 제공하고 있다. 

 ** 가이드라인 작성 시점 기준 잔여 issue로, 20210304 version으로 지정함


In [1]:
import os

gpu_gtg = False
if int(os.environ.get("COLAB_GPU")) > 0:
    gpu_gtg = "COLAB_GPU" in os.environ

tpu_gtg = "COLAB_TPU_ADDR" in os.environ

if tpu_gtg: # tpu
    print("TPU")
    #VERSION = "nightly"

    # https://github.com/pytorch/builder/pull/750
    VERSION = "20210304" # was 20200607" 

    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version $VERSION

###A.1.2. 패키지 설치

 다음과 같이 이그나이트의 최신 version을 설치한다. 현 시점 기준 Colab에서 제공하는 VM은, 이그나이트가 사전 설치되어 있지 않은 상태이기 때문이다.  ~~참고로 pip 명령어는 package installer for python)의 약자이며, 아래 명령문 실행 시 이그나이트의 pre-release를 PyPI(python package index)로부터 설치하게 된다.~~


In [2]:
!pip install --pre pytorch-ignite

     |████████████████████████████████| 233 kB 7.5 MB/s 


In [3]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models

import torchsummary

import ignite
import ignite.distributed as idist
from ignite.engine import Engine, Events, create_supervised_evaluator, create_supervised_trainer
from ignite.metrics import Accuracy, Loss, RunningAverage, ConfusionMatrix
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.utils import setup_logger

In [6]:
from ignite.contrib.handlers import ProgressBar, HandlersTimeProfiler

In [12]:
def training(local_rank, config, **kwargs):
    print("local rank: ", local_rank)

    ###########################################################
    # 데이터 준비
    train_transform = transforms.Compose(
        [
            transforms.Pad(4),
            transforms.RandomCrop(32, fill=128),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ]
    )

    test_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),])

    if idist.get_local_rank() > 0:
        idist.barrier()

    trainset = torchvision.datasets.CIFAR10(root=config["data_path"], train=True, download=True, transform=train_transform)
    testset = torchvision.datasets.CIFAR10(root=config["data_path"], train=False, download=True, transform=test_transform)

    if idist.get_local_rank() == 0:
        idist.barrier()

    trainloader = idist.auto_dataloader(trainset, batch_size=config["batch_size"], shuffle=True, num_workers=config["num_workers"], drop_last=True)
    testloader = idist.auto_dataloader(testset, batch_size=config["batch_size"], shuffle=False, num_workers=config["num_workers"],)

    ###########################################################
    # 모델, 옵티마이저, 로스, 트레이너, 이밸류에이터
    num_classes = 10
    model = models.resnet18(num_classes = num_classes)
       
    model = idist.auto_model(model)
    optimizer = idist.auto_optim(optim.Adam(model.parameters(), lr=0.001))

    criterion = nn.CrossEntropyLoss().to(idist.device())

    trainer = create_supervised_trainer(model, optimizer, criterion, device=idist.device())
    trainer.logger = setup_logger("hkim-trainer")




    #*************************************************************
    # HandlersTimeProfiler
    profiler = HandlersTimeProfiler()
    profiler.attach(trainer)

    # Init and attach progressbar
    pbar = ProgressBar(persist=True)
    pbar.attach(trainer, metric_names="all")




    metrics = {
        'accuracy':Accuracy(),
        'ce':Loss(criterion),
    }

    val_evaluator = create_supervised_evaluator(model, metrics=metrics, device=idist.device())
    val_evaluator.logger = setup_logger("hkim-val_evaluator")

    # track a running average of the scalar loss output for each batch.
    RunningAverage(output_transform=lambda x: x).attach(trainer, 'loss')

    ###########################################################
    # 이벤트

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(trainer):
        state = val_evaluator.run(testloader)
        metrics = val_evaluator.state.metrics
        accuracy = metrics['accuracy']*100
        loss = metrics['ce']
        #log_metrics(val_evaluator.logger, state.epoch, state.times["COMPLETED"], "validation evaluator", state.metrics)
        pbar.log_message("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}".format(state.epoch, accuracy, loss))
        pbar.n = pbar.last_print_n = 0

    trainer.run(trainloader, max_epochs=config["num_epochs"])    


    #*************************************************************
    # HandlersTimeProfiler
    profiler.print_results(profiler.get_results())    

In [13]:
config = {
    "seed": 543,
    "data_path" : "./cifar10",
    "output_path" : "./output-cifar10/",
    "model" : "resnet18",
    "batch_size" : 512,
    "momentum" : 0.9,
    "weight_decay" : 1e-4,
    "num_workers" : 2,
    "num_epochs" : 5,
    "learning_rate" : 0.4,
    "num_warmup_epochs" : 4,
    "validate_every" : 3, 
    "checkpoint_every" : 1000,
    "backend" : None, 
    "resume_from" : None, 
    "log_every_iters" : 15,
    "nproc_per_node" : None, 
    "stop_iteration" : None, 
    "with_amp" : False,
    "log_interval" : 10,
    "verbose_set" : False,
    "verbose_set2" : False,
    "verbose_loader" : False

}

if not (tpu_gtg or gpu_gtg): # cpu
    config["backend"] = 'gloo'
    config["nproc_per_node"] = 8
elif gpu_gtg: # gpu
    config["backend"] = 'nccl'
    config["nproc_per_node"] = 1
elif tpu_gtg: # tpu
    config["backend"] = 'xla-tpu'
    config["nproc_per_node"] = 8
else: # error
    raise RuntimeError("Unknown environment: tpu_gtg {}, gpu_gtg {}".format(tpu_gtg, gpu_gtg))

if config["backend"] == "xla-tpu" and config["with_amp"]:
    raise RuntimeError("The value of with_amp should be False if backend is xla")


dist_configs = {'nproc_per_node': config["nproc_per_node"], "start_method": "fork"}  

#def log_metrics(logger, epoch, elapsed, tag, metrics):
#    metrics_output = "\n".join([f"\t{k}: {v}" for k, v in metrics.items()])
#    logger.info(f"\nEpoch {epoch} - Evaluation time (seconds): {elapsed:.2f} - {tag} metrics:\n {metrics_output}")

with idist.Parallel(backend=config["backend"], **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=1)

2021-09-14 01:11:43,420 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'nccl'
2021-09-14 01:11:43,421 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 1
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-09-14 01:11:43,422 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f178948e950>' in 1 processes


local rank:  0
Files already downloaded and verified
Files already downloaded and verified


2021-09-14 01:11:48,421 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset 'Dataset CIFAR10': 
	{'batch_size': 512, 'shuffle': True, 'num_workers': 2, 'drop_last': True, 'pin_memory': True}
2021-09-14 01:11:48,423 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset 'Dataset CIFAR10': 
	{'batch_size': 512, 'shuffle': False, 'num_workers': 2, 'pin_memory': True}
2021-09-14 01:11:48,592 hkim-trainer INFO: Engine run starting with max_epochs=5.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1/97]   1%|1          [00:00<?]

2021-09-14 01:12:06,257 hkim-val_evaluator INFO: Engine run starting with max_epochs=1.
2021-09-14 01:12:08,127 hkim-val_evaluator INFO: Epoch[1] Complete. Time taken: 00:00:02
2021-09-14 01:12:08,130 hkim-val_evaluator INFO: Engine run complete. Time taken: 00:00:02


...

2021-09-14 01:13:23,844 hkim-val_evaluator INFO: Engine run starting with max_epochs=1.
2021-09-14 01:13:25,652 hkim-val_evaluator INFO: Epoch[1] Complete. Time taken: 00:00:02
2021-09-14 01:13:25,656 hkim-val_evaluator INFO: Engine run complete. Time taken: 00:00:02


Validation Results - Epoch: 1  Avg accuracy: 67.12 Avg loss: 0.96


2021-09-14 01:13:25,715 hkim-trainer INFO: Epoch[5] Complete. Time taken: 00:00:20
2021-09-14 01:13:25,717 hkim-trainer INFO: Engine run complete. Time taken: 00:01:37



--------------------------------------------  -----------------------  --------------  --------------  --------------  --------------  --------------  
Handler                                       Event Name                     Total(s)      Min(s)/IDX      Max(s)/IDX         Mean(s)          Std(s)  
--------------------------------------------  -----------------------  --------------  --------------  --------------  --------------  --------------  
Metric.started                                EPOCH_STARTED                     8e-05         1e-05/0         2e-05/4           2e-05             0.0  
ProgressBar._close                            EPOCH_COMPLETED                  0.0223         0.002/0       0.00668/3         0.00446         0.00171  
training.<locals>.log_validation_results      EPOCH_COMPLETED                 9.47877       1.85624/3       1.93348/0         1.89575         0.03407  
_OutputHandler                                ITERATION_COMPLETED            12.30616  

2021-09-14 01:13:25,954 ignite.distributed.launcher.Parallel INFO: End of run


## License


---


Note: This is not an official [LG AI Research](https://www.lgresearch.ai/) product but sample code provided for an educational purpose

<br/>
author: John H. Kim
<br/>  
email: john.kim@lgresearch.ai / secutron@naver.com  


---